In [ ]:
import torch
import torch.nn as nn

class Model(nn.Module):

  def __init__(self, num_feature):
    super().__init__()
    self.network = nn.Sequential(
        nn.Linear(num_feature, 3),
        nn.ReLU(),
        nn.Linear(3,1),

        nn.Sigmoid()
    )

  def forward(self, feature):
    out = self.network(features)
    return out




In [ ]:
# Creating dataset
features = torch.rand(10,5)

model = Model(features.shape[1])

model(features)


tensor([[0.4974],
        [0.4929],
        [0.4993],
        [0.5171],
        [0.5031],
        [0.5057],
        [0.4888],
        [0.5054],
        [0.5132],
        [0.4954]], grad_fn=<SigmoidBackward0>)

In [ ]:
from torchinfo import summary
summary(model, input_size=(10,5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Sequential: 1-1                        [10, 1]                   --
│    └─Linear: 2-1                       [10, 3]                   18
│    └─ReLU: 2-2                         [10, 3]                   --
│    └─Linear: 2-3                       [10, 1]                   4
│    └─Sigmoid: 2-4                      [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

**Running on breast cancer data**

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:, 0], test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [55]:
X_train_tensor.shape

torch.Size([455, 30])

In [59]:
class MySimpleNN(nn.Module):

  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)
    return out



In [60]:
# Parameters
learning_rate = 0.1
epochs = 25

In [61]:
loss_function = nn.BCELoss()


In [66]:
model = MySimpleNN(X_train_tensor.shape[1])
optimer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):

  y_pred = model(X_train_tensor)
  loss = loss_function(y_pred, y_train_tensor.view(-1, 1))
  optimer.zero_grad()
  loss.backward()
  optimer.step()

  print(f'Epochs: {epoch + 1}, Loss: {loss.item()}')


Epochs: 1, Loss: 1.048506736755371
Epochs: 2, Loss: 0.7190213799476624
Epochs: 3, Loss: 0.5427256226539612
Epochs: 4, Loss: 0.45105189085006714
Epochs: 5, Loss: 0.3961629569530487
Epochs: 6, Loss: 0.3588617146015167
Epochs: 7, Loss: 0.33135315775871277
Epochs: 8, Loss: 0.309935986995697
Epochs: 9, Loss: 0.29261574149131775
Epochs: 10, Loss: 0.27821168303489685
Epochs: 11, Loss: 0.2659740149974823
Epochs: 12, Loss: 0.25540030002593994
Epochs: 13, Loss: 0.24613924324512482
Epochs: 14, Loss: 0.23793664574623108
Epochs: 15, Loss: 0.23060335218906403
Epochs: 16, Loss: 0.22399510443210602
Epochs: 17, Loss: 0.21799972653388977
Epochs: 18, Loss: 0.21252819895744324
Epochs: 19, Loss: 0.2075090855360031
Epochs: 20, Loss: 0.2028840184211731
Epochs: 21, Loss: 0.19860480725765228
Epochs: 22, Loss: 0.19463124871253967
Epochs: 23, Loss: 0.19092947244644165
Epochs: 24, Loss: 0.18747062981128693
Epochs: 25, Loss: 0.1842300146818161


In [67]:
# Evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.5301631093025208
